In [1]:
# Import PuLP modeler functions
from pulp import *
from sympy import *
init_printing(pretty_print = True)

NA = 10000   #An arbitrarily high penalty which means a country can't trade with itself
def reduce(mat): return Matrix(mat.shape[0], mat.shape[1], lambda i,j: None if abs(mat[i,j]) >= NA else mat[i,j])

Countries = ['A', 'B', 'C', 'D']
num = len(Countries)
trade = [[0 for i in range(num)] for j in range(num)]
transport = 0;

iniExports = {'A' : 1000, 'B' : 300, 'C' : 500, 'D' : 800}        #Balanced at a net of 260 units of trade
iniImports = {'A' : 400, 'B' : 800, 'C' : 1000, 'D' : 400}

#Use this table to indicate how many units of trade must occur between certain countries
#Read rows, then columm: Country _ must export _ units to Country _
neededTrades = [# A B C D
                [NA, 100, 200, 200], #A
                [0, NA, 150, 0], #B
                [0, 150, NA, 0], #C
                [300, 0, 0, NA], #D
               ]
#Costs will fluctuate over time, depending on how tariffs impact costs
#Read rows, then columm: Country _ exports to Country _ at tariff _
cost = [#Country imports from
        #A B C D
        [NA, 4, 4, 6], #A   Country
        [3, NA, 4, 4], #B   exports
        [3, 4, NA, 4], #C   to at
        [4, 4, 4, NA], #D   cost
       ]

#This reaugments the exports and imports that cannot be done freely
exports = iniExports.copy()
imports = iniImports.copy()
for i in range(num):
    for j in range(num):
        if neededTrades[i][j] != NA:
            exports[Countries[i]] -= neededTrades[i][j]
            imports[Countries[j]] -= neededTrades[i][j]
            transport += neededTrades[i][j] * cost[i][j]
            
print("Exports available:", iniExports)
print("Imports needed:", iniImports)
display("Costs Matrix:", reduce(Matrix(cost)))
display("Needed Trades Matrix:", reduce(Matrix(neededTrades)))
print("Exports available to freely trade:", exports)
print("Imports needed from free trade:", imports)

Exports available: {'A': 1000, 'B': 300, 'C': 500, 'D': 800}
Imports needed: {'A': 400, 'B': 800, 'C': 1000, 'D': 400}


'Costs Matrix:'

⎡None   4     4     6  ⎤
⎢                      ⎥
⎢ 3    None   4     4  ⎥
⎢                      ⎥
⎢ 3     4    None   4  ⎥
⎢                      ⎥
⎣ 4     4     4    None⎦

'Needed Trades Matrix:'

⎡None  100   200   200 ⎤
⎢                      ⎥
⎢ 0    None  150    0  ⎥
⎢                      ⎥
⎢ 0    150   None   0  ⎥
⎢                      ⎥
⎣300    0     0    None⎦

Exports available to freely trade: {'A': 500, 'B': 150, 'C': 350, 'D': 500}
Imports needed from free trade: {'A': 100, 'B': 550, 'C': 650, 'D': 200}


In [2]:
def tradeMatrix(cost,exports,imports):
    #The following segment utilizes the PuLP solver to find an optimal solution            
    # The cost data is made into a dictionary
    costs = makeDict([exports,imports],cost,0)

    # Creates the 'prob' variable to contain the problem data
    prob = LpProblem("Trade War Problem",LpMinimize)

    # Creates a list of tuples containing all the possible routes for transport
    Routes = [(e,i) for e in exports for i in imports]
    vars = LpVariable.dicts("Route",(exports,imports),0,None,LpInteger)

    # The objective function is added to 'prob' first
    prob += lpSum([vars[e][i]*costs[e][i] for (e,i) in Routes]), "Sum_of_Transporting_Costs"

    # The supply maximum constraints are added to prob for each supply node
    for e in exports:
        prob += lpSum([vars[e][i] for i in imports])<=exports[e], "Sum_of_exports_%s"%e

    # The demand minimum constraints are added to prob for each demand node
    for i in imports:
        prob += lpSum([vars[e][i] for e in exports])>=imports[i], "Sum_of_imports_%s"%i

    prob.writeLP("TradeWarModel.lp")
    prob.solve()

    return prob

In [3]:
TD = tradeMatrix(cost,exports,imports)

# The status of the solution is printed to the screen
print("Status:", LpStatus[TD.status])

for v in TD.variables():
    trade[Countries.index(v.name[6])][Countries.index(v.name[8])] = v.varValue
finalTrade = reduce(Matrix(trade) + Matrix(neededTrades))

display("Final trade distribution Matrix:", finalTrade)
print("Cost of Needed Trading: ", transport)
print("Cost of Free Trading: ", int(value(TD.objective)))

Status: Optimal


'Final trade distribution Matrix:'

⎡None   600.0  200.0  200.0⎤
⎢                          ⎥
⎢  0    None   300.0    0  ⎥
⎢                          ⎥
⎢100.0  200.0  None   200.0⎥
⎢                          ⎥
⎣300.0    0    500.0  None ⎦

Cost of Needed Trading:  4800
Cost of Free Trading:  5900


In [4]:
def CountryExpenses(FT,cost):
    exCost = [0 for i in range(num)]
    imCost = [0 for i in range(num)]
    for i in range(num):
        for j in range(num):
            if FT[i,j] != None:
                exCost[i] += FT[i,j]*cost[i][j]
                imCost[j] += FT[i,j]*cost[i][j]
    exp = dict(zip(Countries, [int(i) for i in exCost]))
    imp = dict(zip(Countries, [int(i) for i in imCost]))
    return exp,imp, exCost,imCost

CE = CountryExpenses(finalTrade,cost)
print('Cost spent on Exports: ', CE[0])
print('Cost earnt from Imports: ', CE[1])

Cost spent on Exports:  {'A': 4400, 'B': 1200, 'C': 1900, 'D': 3200}
Cost earnt from Imports:  {'A': 1500, 'B': 3200, 'C': 4000, 'D': 2000}
